In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


train = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/train.csv')
transactions = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/transactions.csv')
test = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/test.csv')
holidays = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/holidays_events.csv')
oil = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/oil.csv')
stores = pd.read_csv('C:/Users/akshi/coding/projects/dsoc/week2/data/stores.csv')



In [7]:
# Merge train data with store information
train = train.merge(stores, on='store_nbr', how='left')

# Merge train data with transactions
train = train.merge(transactions, on=['date', 'store_nbr'], how='left')

# Merge train data with oil prices
train = train.merge(oil, on='date', how='left')

# Merge train data with holidays
train = train.merge(holidays, on='date', how='left')

# Handle missing values if any
train.fillna(method='ffill', inplace=True)

# One-hot encode categorical variables
train = pd.get_dummies(train, columns=['store_nbr', 'type_x', 'type_y', 'locale', 'locale_name', 'description', 'transferred'])

# Convert 'date' column to datetime
train['date'] = pd.to_datetime(train['date'])

# Extract date-based features
train['day_of_week'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['day_of_year'] = train['date'].dt.dayofyear
train['week_of_year'] = train['date'].dt.isocalendar().week
train['quarter'] = train['date'].dt.quarter

C:\Users\akshi\AppData\Local\Temp\ipykernel_19352\3057108874.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method='ffill', inplace=True)
C:\Users\akshi\AppData\Local\Temp\ipykernel_19352\3057108874.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.fillna(method='ffill', inplace=True)


In [8]:
# Select numerical features to scale
from sklearn.preprocessing import StandardScaler
numerical_features = ['transactions', 'dcoilwtico', 'day_of_year', 'week_of_year', 'quarter']
scaler = StandardScaler()
train[numerical_features] = scaler.fit_transform(train[numerical_features])

# Check the columns in the train DataFrame
print("Columns in train DataFrame:", train.columns)

Columns in train DataFrame: Index(['id', 'date', 'family', 'sales', 'onpromotion', 'city', 'state',
       'cluster', 'transactions', 'dcoilwtico',
       ...
       'description_Traslado Primer dia del ano', 'description_Viernes Santo',
       'transferred_False', 'transferred_True', 'day_of_week', 'month', 'year',
       'day_of_year', 'week_of_year', 'quarter'],
      dtype='object', length=211)


In [9]:
# Set 'date' as the index for ARIMA
train.set_index('date', inplace=True)

# Use sales data for ARIMA
sales_data = train['sales']

# Fit the ARIMA model
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(sales_data, order=(5, 1, 0))
model_fit = model.fit()

# Print the summary of the model
print(model_fit.summary())


# Fit the SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(sales_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model_fit = model.fit()

# Print the summary of the model
print(model_fit.summary())



C:\Users\akshi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\akshi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\akshi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
# Install Prophet if you haven't already
# !pip install prophet

from prophet import Prophet

# Prepare data for Prophet
prophet_data = train.reset_index()[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})

# Fit Prophet model
model_prophet = Prophet()
model_prophet.fit(prophet_data)

# Make future predictions
future = model_prophet.make_future_dataframe(periods=365)  # Forecasting for 1 year into the future
forecast = model_prophet.predict(future)

# Plot the forecast
fig = model_prophet.plot(forecast)
plt.title('Prophet Forecast for Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()


In [ ]:
# Importing necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Prepare data for LSTM
lstm_data = train.reset_index()[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(lstm_data[['y']])

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Set sequence length
sequence_length = 10

# Create sequences for LSTM
X, y = create_sequences(scaled_data, sequence_length)

# Reshape X to fit LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(units=1))

# Compile the model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model_lstm.fit(X, y, epochs=50, batch_size=32)

# Predict using LSTM model
forecast_lstm = model_lstm.predict(X[-365:])  # Forecasting for 1 year into the future

# Inverse transform the forecasted data
forecast_lstm = scaler.inverse_transform(forecast_lstm)

# Plot the LSTM forecast
plt.figure(figsize=(10, 6))
plt.plot(lstm_data['ds'], lstm_data['y'], label='Actual Sales')
plt.plot(lstm_data['ds'][-365:], forecast_lstm, label='LSTM Forecast')
plt.title('LSTM Forecast for Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()
